In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [ ]:
indie_df = pd.read_csv('Data/indiegogo_2.csv')

In [ ]:
#create target value column, based on funded_percent
indie_df['funded_percent'] = indie_df['funded_percent'].str.replace('%', '').astype(float)
indie_df['is_success'] = (indie_df['funded_percent'] >= 80).astype(int)

In [ ]:
#add day of the week column, based on launch date
from datetime import datetime
indie_df['date_launch'] = pd.to_datetime(indie_df['date_launch'])
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
indie_df['day_of_week'] = indie_df['date_launch'].apply(lambda x: day_names[x.weekday()])

In [ ]:
#dummifying 
day_dummies = pd.get_dummies(indie_df['day_of_week'])
indie_df = pd.concat([indie_df, day_dummies], axis=1)
indie_df.columns = indie_df.columns.str.lower()

In [ ]:
#indie_df = pd.to_csv('Data/indiepygo_ready.csv') #save pdf

In [13]:
df = pd.read_csv('Data/IndiePyGo_nulldropped.csv')

In [14]:
indie_text_df = df[['title', 'tagline', 'is_success']]
indie_text_df

,title,tagline,is_success
0,Join the Electric Revolution!!!,Pure electric motorcycle proves a powerful alt...,0
1,Relief Trip to Haiti,Send Me to Haiti...I'm needed there!,0
2,"Out To Reach Leogane, Haiti 2010",Haiti Relief Mission to Leogane,0
3,The Transpersonal Papers: 1861-2010,My third book on Fezziwig Press.,0
4,Homeless Veterans need a Hand UP not Hand Out!,Homeless Veterans Transitional Housing Develop...,0
...,...,...,...
20613,Totally Gay Productions,Trans filmmaker making queer documentaries. It...,0
20614,Lady Crow debut EP,"Help us finance Every Stone, our first EP!",0
20615,"JOSA, the ultimate Venice-Inspired bracelet","Benefits Save Venice, helps to protect Venice'...",0
20616,My Campaign Title,Hi,0


In [15]:
indie_text_df['text'] = indie_text_df['title'] + ' ' + indie_text_df['tagline']
indie_text_df['text']

/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_62190/2641864021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indie_text_df['text'] = indie_text_df['title'] + ' ' + indie_text_df['tagline']


0        Join the Electric Revolution!!! Pure electric ...
1        Relief Trip to Haiti Send Me to Haiti...I'm ne...
2        Out To Reach Leogane, Haiti 2010 Haiti Relief ...
3        The Transpersonal Papers: 1861-2010 My third b...
4        Homeless Veterans need a Hand UP not Hand Out!...
                               ...                        
20613    Totally Gay Productions Trans filmmaker making...
20614    Lady Crow debut EP Help us finance Every Stone...
20615    JOSA, the ultimate Venice-Inspired bracelet Be...
20616                                 My Campaign Title Hi
20617                                 My Campaign Title Hi
Name: text, Length: 20618, dtype: object

In [16]:
indie_text_df.drop(['title', 'tagline'], axis=1, inplace=True)


/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_62190/1899599088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indie_text_df.drop(['title', 'tagline'], axis=1, inplace=True)


In [17]:
indie_text_df

,is_success,text
0,0,Join the Electric Revolution!!! Pure electric ...
1,0,Relief Trip to Haiti Send Me to Haiti...I'm ne...
2,0,"Out To Reach Leogane, Haiti 2010 Haiti Relief ..."
3,0,The Transpersonal Papers: 1861-2010 My third b...
4,0,Homeless Veterans need a Hand UP not Hand Out!...
...,...,...
20613,0,Totally Gay Productions Trans filmmaker making...
20614,0,Lady Crow debut EP Help us finance Every Stone...
20615,0,"JOSA, the ultimate Venice-Inspired bracelet Be..."
20616,0,My Campaign Title Hi


In [18]:
indie_text_df.to_csv('indiepygo_text_fornlp.csv', index=False)